# Study 1 Analysis
Code for analyzing data from the survey in Study 1

# Load packages

In [50]:
import os
import copy
import json
import csv
from datetime import datetime

import pandas as pd
from scipy.stats import mannwhitneyu

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)
pd.options.mode.chained_assignment = None  # default='warn'

# Load Data

In [51]:
survey_data_df = pd.read_csv(
    "./data/final-survey-data_04-13-25.csv",
    keep_default_na=False,
)
print(json.dumps(list(survey_data_df.columns), indent=4))
survey_data_df.head()

[
    "Timestamp",
    "Please select your age",
    "Do you identify as blind or low vision?",
    "Do you primarily use screen readers to access digital content on a computer or smartphone?",
    "Do you speak English?",
    "Do you currently reside in the United States?",
    "How often do you use remote sighted visual interpreting applications (e.g., Aira, Be My Eyes)?",
    "How often do you use automated or AI based visual interpreting applications (e.g., Microsoft Seeing AI, Be My AI, TapTapSee, Access AI)?",
    "How often do you use general purpose AI applications (e.g., ChatGPT, Google Gemini, Claude AI) for understanding visual information?",
    "Which of the following AI based tools have you used to automatically generate visual captions of your environment, objects, or images? Select all that apply.",
    "Which of the following tasks have you used AI generated captioning for? Select all that apply.",
    "What kinds of tasks do you feel AI generated captions are most app

,Timestamp,Please select your age,Do you identify as blind or low vision?,Do you primarily use screen readers to access digital content on a computer or smartphone?,Do you speak English?,Do you currently reside in the United States?,"How often do you use remote sighted visual interpreting applications (e.g., Aira, Be My Eyes)?","How often do you use automated or AI based visual interpreting applications (e.g., Microsoft Seeing AI, Be My AI, TapTapSee, Access AI)?","How often do you use general purpose AI applications (e.g., ChatGPT, Google Gemini, Claude AI) for understanding visual information?","Which of the following AI based tools have you used to automatically generate visual captions of your environment, objects, or images? Select all that apply.",Which of the following tasks have you used AI generated captioning for? Select all that apply.,What kinds of tasks do you feel AI generated captions are most appropriate for?,What kinds of tasks do you feel AI generated captions are not appropriate for?,Which of the following AI based tools have you used to identify or understand products? Select all that apply.,Are there any other AI based tools that you use to identify and understand products?,How often do you use AI based tools to identify or understand products?,"How often do you use remote sighted visual interpreting applications, such as Aira or Be My Eyes, or other human assistance to identify or understand products?",Which of the following are you most likely to use when searching for a specific product item at a physical store?,Which of the following are you most likely to use when browsing products at a physical store?,Which of the following are you most likely to use when wanting to compare the details of two products side by side?,Which of the following are you most likely to use when identifying an unknown item in your home?,Which of the following are you most likely to use when reading a label on a food item?,Which of the following are you most likely to use when reading a label on medication?,Which of the following are you most likely to use when identifying personal care products or toiletries?,Which of the following are you most likely to use when checking expiration dates on products?,Which of the following are you most likely to use when checking allergen information on products?,Are there any other cases where you would prefer to use an AI tool to identify or understand products rather than human sighted assistance?,Are there any other cases where you would prefer to have human sighted assistance identify or understand products rather than an AI tool?,Which of the following are you most likely to use when efficiency in identifying and understanding a product is your top concern?,"Do you have any thoughts related to efficiency tradeoffs when using AI tools versus human assistance to identify and understand products? If so, please explain.","Which of the following are you most likely to use when data privacy in identifying and understanding a product is your top concern (e.g., the extent to which data you share is kept private)?","Do you have any thoughts related to data privacy tradeoffs when using AI tools versus human assistance to identify and understand products? If so, please explain.","Which of the following are you most likely to use when personal privacy in identifying and understanding a product is your top concern (e.g., for fear of embarrassment or stigma)?","Do you have any thoughts related to personal privacy tradeoffs when using AI tools versus human assistance to identify and understand products? If so, please explain.",Which of the following are you most likely to use when accuracy in identifying and understanding a product is your top concern?,"Do you have any thoughts related to accuracy tradeoffs when using AI tools versus human assistance to identify and understand products? If so, please explain.",Which of the following are you most likely to use when financial cost in identifying and und

In [52]:
confidence_col = "Again, think about the AI tool you use most to caption products. When the tool says a photo is not good enough to caption or returns a similar error, how confident are you in knowing why the photo is not good enough?"


survey_data_df[survey_data_df[confidence_col] != ""][confidence_col].map(
    {
        "Extremely confident": 5,
        "Very confident": 4,
        "Somewhat confident": 3,
        "Slightly confident": 2,
        "Not at all confident": 1,
    }
).describe()

count    79.000000
mean      2.392405
std       0.979504
min       1.000000
25%       2.000000
50%       3.000000
75%       3.000000
max       5.000000
Name: Again, think about the AI tool you use most to caption products. When the tool says a photo is not good enough to caption or returns a similar error, how confident are you in knowing why the photo is not good enough?, dtype: float64

# Statistical analysis 

In [53]:
image_quality_factor_cols = [
    "Keeping the tool you use most in mind, how much does lighting in your environment affect the quality of AI generated captions for products?",
    "How much does taking clear, non-blurry photos affect the quality of AI generated captions for products?",
    "How much does having the object in full view of your camera, rather than a partial view of the object, affect the quality of AI generated captions for products?",
    "How much does rotating your camera or the object affect the quality of AI generated captions for products?",
    "How much does your hand placement or how you are holding the object affect the quality of AI generated captions for products?",
    "How much does moving your camera closer or further from an object affect the quality of AI generated captions for products?",
    "How much does a product being uncommon or unique affect the quality of AI generated captions?",
]

image_assessment_factor_cols = [
    "How well does the AI tool help you assess lighting conditions when taking a photo?",
    "How well does the AI tool help you assess whether the photo you took is clear and non-blurry?",
    "How well does the AI tool help you assess whether an object is in full view of your camera when taking photos?",
    "How well does the AI tool help you assess object orientation when taking photos?",
    "How well does the AI tool help you understand your hand positioning or placement relative to the object you want to photograph?",
    "How well does the AI tool help you assess the distance between your camera and the object of interest when taking photos?",
    "How well does the AI tool help you assess whether a product you are taking a photo of is uncommon or unique?",
]


mapping_dict = {
    "To a great extent": 5,
    "Somewhat": 4,
    "Very little": 3,
    "Not at all": 2,
    "I am not sure": 1,
}

In [54]:
condition_col = "Which of the following AI based tools do you use the most to identify and understand products?"
survey_data_df[condition_col].value_counts()

Which of the following AI based tools do you use the most to identify and understand products?
Be My AI (part of Be My Eyes that does not involve human visual intepreters)                30
Microsoft Seeing AI                                                                         30
ChatGPT                                                                                      6
Ray-Ban Meta Glasses                                                                         6
AI captioning features built into a screen reader, such as with VoiceOver, NVDA, or JAWS     4
Access AI (part of Aira that does not involve human visual interpreters)                     3
Alexa show                                                                                   1
Microsoft Co-Pilot                                                                           1
None                                                                                         1
TapTapSee                                         

In [55]:
len(survey_data_df)

86

In [56]:
# calculate mean and statistical differences for how much image quality affects response
for question in image_quality_factor_cols:
    # create dataframes for each condition
    be_my_ai_products_df = survey_data_df[
        survey_data_df[condition_col]
        == "Be My AI (part of Be My Eyes that does not involve human visual intepreters)"
    ]
    seeing_ai_products_df = survey_data_df[
        survey_data_df[condition_col] == "Microsoft Seeing AI"
    ]

    # convert to number
    seeing_ai_products_df[question] = seeing_ai_products_df[question].map(mapping_dict)
    be_my_ai_products_df[question] = be_my_ai_products_df[question].map(mapping_dict)

    # remove rows where any of the image quality factor columns are empty
    seeing_ai_products_df = seeing_ai_products_df[
        seeing_ai_products_df[image_quality_factor_cols].notna().all(axis=1)
    ]
    be_my_ai_products_df = be_my_ai_products_df[
        be_my_ai_products_df[image_quality_factor_cols].notna().all(axis=1)
    ]

    print(
        f"Be My AI: n = {len(be_my_ai_products_df)} | Seeing AI: n = {len(seeing_ai_products_df)}"
    )

    U, p = mannwhitneyu(
        seeing_ai_products_df[question],
        be_my_ai_products_df[question],
        nan_policy="omit",
    )
    print(f"{question}", end="")
    if p <= 0.05:
        print(" -- STATISTICALLY SIGNIFICANT")
    else:
        print()
    print(
        f"Mean Seeing AI: {seeing_ai_products_df[question].mean():.2f} | Mean Be My AI: {be_my_ai_products_df[question].mean():.2f}"
    )
    print(f"U: {U}, p: {p:.4f}", end="")
    if p < 0.001:
        print("***", end="")
    elif p < 0.01:
        print("**", end="")
    elif p < 0.05:
        print("*", end="")
    elif p < 0.1:
        print(".", end="")
    print("\n")

Be My AI: n = 30 | Seeing AI: n = 30
Keeping the tool you use most in mind, how much does lighting in your environment affect the quality of AI generated captions for products?
Mean Seeing AI: 3.97 | Mean Be My AI: 3.77
U: 467.5, p: 0.7771

Be My AI: n = 30 | Seeing AI: n = 30
How much does taking clear, non-blurry photos affect the quality of AI generated captions for products?
Mean Seeing AI: 4.23 | Mean Be My AI: 4.27
U: 483.0, p: 0.5899

Be My AI: n = 30 | Seeing AI: n = 30
How much does having the object in full view of your camera, rather than a partial view of the object, affect the quality of AI generated captions for products? -- STATISTICALLY SIGNIFICANT
Mean Seeing AI: 4.53 | Mean Be My AI: 4.17
U: 587.5, p: 0.0202*

Be My AI: n = 30 | Seeing AI: n = 30
How much does rotating your camera or the object affect the quality of AI generated captions for products?
Mean Seeing AI: 3.90 | Mean Be My AI: 3.47
U: 513.5, p: 0.3195

Be My AI: n = 30 | Seeing AI: n = 30
How much does you

### Differences in Percieved Impact between Be My AI and Seeing AI 

In [57]:
# create dataframes for each condition
be_my_ai_products_df = survey_data_df[
    survey_data_df[condition_col]
    == "Be My AI (part of Be My Eyes that does not involve human visual intepreters)"
]
seeing_ai_products_df = survey_data_df[
    survey_data_df[condition_col] == "Microsoft Seeing AI"
]

for question in image_quality_factor_cols:
    # convert to number
    seeing_ai_products_df[question] = seeing_ai_products_df[question].map(mapping_dict)
    be_my_ai_products_df[question] = be_my_ai_products_df[question].map(mapping_dict)

# remove rows where any of the image quality factor columns are empty
seeing_ai_products_df = seeing_ai_products_df[
    seeing_ai_products_df[image_quality_factor_cols].notna().all(axis=1)
]
be_my_ai_products_df = be_my_ai_products_df[
    be_my_ai_products_df[image_quality_factor_cols].notna().all(axis=1)
]

# print sample size
print(
    f"Be My AI: n = {len(be_my_ai_products_df)} | Seeing AI: n = {len(seeing_ai_products_df)}"
)

# calculate mean and statistical differences for how much the tool supports diagnosing issues
for question in image_quality_factor_cols:
    U, p = mannwhitneyu(
        seeing_ai_products_df[question],
        be_my_ai_products_df[question],
        nan_policy="omit",
    )
    print(f"{question}", end="")
    if p <= 0.05:
        print(" -- STATISTICALLY SIGNIFICANT")
    else:
        print()
    print(
        f"Mean Seeing AI: {seeing_ai_products_df[question].mean():.2f} | Mean Be My AI: {be_my_ai_products_df[question].mean():.2f}"
    )
    print(f"p: {p:.4f}, U: {U}", end="")
    if p < 0.001:
        print("***", end="")
    elif p < 0.01:
        print("**", end="")
    elif p < 0.05:
        print("*", end="")
    elif p < 0.1:
        print(".", end="")
    print("\n")

Be My AI: n = 30 | Seeing AI: n = 30
Keeping the tool you use most in mind, how much does lighting in your environment affect the quality of AI generated captions for products?
Mean Seeing AI: 3.97 | Mean Be My AI: 3.77
p: 0.7771, U: 467.5

How much does taking clear, non-blurry photos affect the quality of AI generated captions for products?
Mean Seeing AI: 4.23 | Mean Be My AI: 4.27
p: 0.5899, U: 483.0

How much does having the object in full view of your camera, rather than a partial view of the object, affect the quality of AI generated captions for products? -- STATISTICALLY SIGNIFICANT
Mean Seeing AI: 4.53 | Mean Be My AI: 4.17
p: 0.0202, U: 587.5*

How much does rotating your camera or the object affect the quality of AI generated captions for products?
Mean Seeing AI: 3.90 | Mean Be My AI: 3.47
p: 0.3195, U: 513.5

How much does your hand placement or how you are holding the object affect the quality of AI generated captions for products?
Mean Seeing AI: 3.80 | Mean Be My AI: 3

### Differences in Assessment Ability between Be My AI and Seeing AI

In [58]:
# create dataframes for each condition
be_my_ai_products_df = survey_data_df[
    survey_data_df[condition_col]
    == "Be My AI (part of Be My Eyes that does not involve human visual intepreters)"
]
seeing_ai_products_df = survey_data_df[
    survey_data_df[condition_col] == "Microsoft Seeing AI"
]

for question in image_assessment_factor_cols:
    # convert to number
    seeing_ai_products_df[question] = seeing_ai_products_df[question].map(mapping_dict)
    be_my_ai_products_df[question] = be_my_ai_products_df[question].map(mapping_dict)

# remove rows where any of the image quality factor columns are empty
seeing_ai_products_df = seeing_ai_products_df[
    seeing_ai_products_df[image_assessment_factor_cols].notna().all(axis=1)
]
be_my_ai_products_df = be_my_ai_products_df[
    be_my_ai_products_df[image_assessment_factor_cols].notna().all(axis=1)
]

# print sample size
print(
    f"Be My AI: n = {len(be_my_ai_products_df)} | Seeing AI: n = {len(seeing_ai_products_df)}"
)

# calculate mean and statistical differences for how much the tool supports diagnosing issues
for question in image_assessment_factor_cols:
    U, p = mannwhitneyu(
        seeing_ai_products_df[question],
        be_my_ai_products_df[question],
        nan_policy="omit",
    )
    print(f"{question}", end="")
    if p <= 0.05:
        print(" -- STATISTICALLY SIGNIFICANT")
    else:
        print()
    print(
        f"Mean Seeing AI: {seeing_ai_products_df[question].mean():.2f} | Mean Be My AI: {be_my_ai_products_df[question].mean():.2f}"
    )
    print(f"p: {p:.4f}, U: {U}", end="")
    if p < 0.001:
        print("***", end="")
    elif p < 0.01:
        print("**", end="")
    elif p < 0.05:
        print("*", end="")
    elif p < 0.1:
        print(".", end="")
    print("\n")

Be My AI: n = 26 | Seeing AI: n = 28
How well does the AI tool help you assess lighting conditions when taking a photo?
Mean Seeing AI: 2.75 | Mean Be My AI: 3.19
p: 0.1524, U: 283.5

How well does the AI tool help you assess whether the photo you took is clear and non-blurry? -- STATISTICALLY SIGNIFICANT
Mean Seeing AI: 3.11 | Mean Be My AI: 3.69
p: 0.0342, U: 245.5*

How well does the AI tool help you assess whether an object is in full view of your camera when taking photos?
Mean Seeing AI: 3.61 | Mean Be My AI: 3.54
p: 0.8630, U: 374.0

How well does the AI tool help you assess object orientation when taking photos?
Mean Seeing AI: 3.29 | Mean Be My AI: 3.54
p: 0.3769, U: 315.5

How well does the AI tool help you understand your hand positioning or placement relative to the object you want to photograph?
Mean Seeing AI: 2.68 | Mean Be My AI: 3.00
p: 0.1225, U: 279.5

How well does the AI tool help you assess the distance between your camera and the object of interest when taking ph